Copyright 2020 Konstantin Yakovlev, Matthias Anderer

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

# Imports & Functions

In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
import pathlib
import lightgbm as lgb

# custom imports
from multiprocessing import Pool        # Multiprocess Runs
warnings.filterwarnings('ignore')

In [2]:
########################### Helpers
#################################################################################
## Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)


## Loss func hyperparamter: LOSS_MUTIPLIER

In [3]:
LOSS_MULTIPLIER = 1.0 # Set multiplier according to desired under-/overshooting

In [4]:
# define custom loss function
def custom_asymmetric_train(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    grad = np.where(residual < 0, -2 * residual, -2 * residual * LOSS_MULTIPLIER)
    hess = np.where(residual < 0, 2, 2 * LOSS_MULTIPLIER)
    return grad, hess

# define custom evaluation metric
def custom_asymmetric_valid(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    loss = np.where(residual < 0, (residual ** 2) , (residual ** 2) * LOSS_MULTIPLIER) 
    return "custom_asymmetric_eval", np.mean(loss), False

In [5]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    df = df[df['store_id']==store]
    
    ############
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle(output_parent/f'test_{store_id}.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


# Customized Variables Def

In [6]:
########################### Model params
#################################################################################
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'tweedie',
        'tweedie_variance_power': 1.1,
        'metric':'rmse',
        'n_jobs': -1,
        'seed': 42,
        'learning_rate': 0.2,
        'bagging_fraction': 0.85,
        'bagging_freq': 1, 
        'colsample_bytree': 0.85,
        'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        'lambda_l1': 0.5,
        'lambda_l2': 0.5
}

In [7]:
########################### Vars
#################################################################################
VER = 1                          # Our model version
SEED = 42                        # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913+28            # End day of our train set
P_HORIZON   = 28                 # Prediction horizon
USE_AUX     = False               # Use or not pretrained models

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]

#PATHS for Features
ORIGINAL = pathlib.Path("./input/data")
input_parent = pathlib.Path("./input/fe_out")
BASE     = input_parent/'grid_part_1.pkl'
PRICE    = input_parent/'grid_part_2.pkl'
CALENDAR = input_parent/'grid_part_3.pkl'
output_parent = pathlib.Path("./input/bottom_out")

#STORES ids
STORES_IDS = pd.read_csv(ORIGINAL/'sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())
print("stores ids are:", STORES_IDS)

stores ids are: ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']


# Train Model

In [8]:
########################### Train Models
#################################################################################
for store_id in STORES_IDS:

    model_name = f'lgb_model_{store_id}_v{VER}.bin'
    if os.path.isfile(output_parent/model_name): 
        continue 
    print('Train {}...'.format(store_id))
    
    # Get grid for current store: 1min
    # features_columns is ['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 
    # 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 
    # 'price_min', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 
    # 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 
    # 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end']
    grid_df, features_columns = get_data_by_store(store_id)
    
    # Masks for 
    # Train (All data less than 1913)
    # "Validation" (Last 28 days - not real validatio set)
    # Test (All data greater than 1913 day, 
    #       with some gap for recursive features)
    train_mask = grid_df['d'] <= END_TRAIN
    valid_mask = train_mask & (grid_df['d'] > (END_TRAIN - P_HORIZON))
    preds_mask = grid_df['d'] > (END_TRAIN - 100)
    
    # Apply masks and save lgb dataset as bin
    # to reduce memory spikes during dtype convertations
    # https://github.com/Microsoft/LightGBM/issues/1032
    # "To avoid any conversions, you should always use np.float32"
    # or save to bin before start training
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                       label=grid_df[train_mask][TARGET])
    train_data.save_binary(output_parent/'train_data.bin')
    train_data = lgb.Dataset(output_parent/'train_data.bin')
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])
    
    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle(output_parent/f'test_{store_id}.pkl')
    del grid_df
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          num_boost_round = 3600, # =n_estimators = num_trees
                          early_stopping_rounds = 50, 
                          valid_sets = [train_data, valid_data],
                          verbose_eval = 100,
                          fobj = custom_asymmetric_train
                          )
    
    # Save model - it's not real '.bin' but a pickle file
    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    pickle.dump(estimator, open(output_parent/model_name, 'wb'))

    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    pickle.dump(estimator, open(output_parent/model_name, 'wb'))
    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    model_name = f'lgb_model_{store_id}_v{VER}.bin'
    pickle.dump(estimator, open(output_parent/model_name, 'wb'))
    del train_data, valid_data, estimator
    gc.collect()
    
    # "Keep" models features for predictions
    MODEL_FEATURES = features_columns

In [18]:
# have a look at training data 
for store_id in STORES_IDS:
    grid_df, features_columns = get_data_by_store(store_id)
    train_mask = grid_df['d'] <= END_TRAIN
    valid_mask = train_mask & (grid_df['d'] > (END_TRAIN - P_HORIZON))
    preds_mask = grid_df['d'] > (END_TRAIN - 100)
    MODEL_FEATURES = features_columns
    break 
print('train, valid and preds dates are:\ntrain: {}\nvalid: {}\npreds: {}'.format(
    grid_df.loc[train_mask, 'd'].unique(), grid_df.loc[valid_mask, 'd'].unique(), grid_df.loc[preds_mask, 'd'].unique()
))
display(grid_df.loc[train_mask]) 
grid_df.loc[train_mask].shape 

train, valid and preds dates are:
train: [  29   30   31 ... 1939 1940 1941]
valid: [1914 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925 1926 1927
 1928 1929 1930 1931 1932 1933 1934 1935 1936 1937 1938 1939 1940 1941]
preds: [1842 1843 1844 1845 1846 1847 1848 1849 1850 1851 1852 1853 1854 1855
 1856 1857 1858 1859 1860 1861 1862 1863 1864 1865 1866 1867 1868 1869
 1870 1871 1872 1873 1874 1875 1876 1877 1878 1879 1880 1881 1882 1883
 1884 1885 1886 1887 1888 1889 1890 1891 1892 1893 1894 1895 1896 1897
 1898 1899 1900 1901 1902 1903 1904 1905 1906 1907 1908 1909 1910 1911
 1912 1913 1914 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925
 1926 1927 1928 1929 1930 1931 1932 1933 1934 1935 1936 1937 1938 1939
 1940 1941 1942 1943 1944 1945 1946 1947 1948 1949 1950 1951 1952 1953
 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965 1966 1967
 1968 1969]


,id,d,sales,item_id,dept_id,cat_id,release,sell_price,price_max,price_min,...,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end
0,HOBBIES_1_008_CA_1_evaluation,29,0.0,HOBBIES_1_008,HOBBIES_1,HOBBIES,0,0.419922,0.500000,0.419922,...,0,0,0,26,8,2,0,4,5,1
1,HOBBIES_1_009_CA_1_evaluation,29,8.0,HOBBIES_1_009,HOBBIES_1,HOBBIES,0,1.559570,1.769531,1.559570,...,0,0,0,26,8,2,0,4,5,1
2,HOBBIES_1_010_CA_1_evaluation,29,0.0,HOBBIES_1_010,HOBBIES_1,HOBBIES,0,3.169922,3.169922,2.970703,...,0,0,0,26,8,2,0,4,5,1
3,HOBBIES_1_012_CA_1_evaluation,29,1.0,HOBBIES_1_012,HOBBIES_1,HOBBIES,0,5.980469,6.519531,5.980469,...,0,0,0,26,8,2,0,4,5,1
4,HOBBIES_1_015_CA_1_evaluation,29,16.0,HOBBIES_1_015,HOBBIES_1,HOBBIES,0,0.720215,0.720215,0.680176,...,0,0,0,26,8,2,0,4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4751344,FOODS_3_823_CA_1_evaluation,1941,2.0,FOODS_3_823,FOODS_3,FOODS,127,2.980469,2.980469,2.480469,...,0,0,0,22,20,5,5,4,6,1
4751345,FOODS_3_824_CA_1_evaluation,1941,0.0,FOODS_3_824,FOODS_3,FOODS,0,2.480469,2.679688,2.470703,...,0,0,0,22,20,5,5,4,6,1
4751346,FOODS_3_825_CA_1_evaluation,1941,1.0,FOODS_3_825,FOODS_3,FOODS,1,3.980469,4.378906,3.980469,...,0,0,0,22,20,5,5,4,6,1
4751347,FOODS_3_826_CA_1_evaluation,1941,1.0,FOODS_3_826,FOODS_3,FOODS,211,1.280273,1.280273,1.280273,...,0,0,0,22,20,5,5,4,6,1


(4751349, 32)

# Predict

In [10]:
########################### Predict
#################################################################################

# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()
print("orginal base_test by combining all test_storeid.pkl")
display(base_test)

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day
for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
        
    for store_id in STORES_IDS:
        
        # Read all our models and make predictions
        # for each day/store pairs
        model_name = f'lgb_model_{store_id}_v{VER}.bin' 
        if USE_AUX: # use pretrained models 
            model_name = AUX_MODEL + model_name
        
        estimator = pickle.load(open(output_parent/model_name, 'rb'))
        
        day_mask = base_test['d']==(END_TRAIN + PREDICT_DAY)
        store_mask = base_test['store_id']==store_id
        
        mask = (day_mask) & (store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
    
    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id', f'F{PREDICT_DAY}']
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
print("After collection of predictions per stores and per")
display(all_preds)

orginal base_test by combining all test_storeid.pkl


,id,d,sales,item_id,dept_id,cat_id,release,sell_price,price_max,price_min,...,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end,store_id
0,HOBBIES_1_001_CA_1_evaluation,1842,4.0,HOBBIES_1_001,HOBBIES_1,HOBBIES,224,8.257812,9.578125,8.257812,...,1,0,13,6,2,5,2,5,1,CA_1
1,HOBBIES_1_002_CA_1_evaluation,1842,0.0,HOBBIES_1_002,HOBBIES_1,HOBBIES,20,3.970703,3.970703,3.970703,...,1,0,13,6,2,5,2,5,1,CA_1
2,HOBBIES_1_003_CA_1_evaluation,1842,1.0,HOBBIES_1_003,HOBBIES_1,HOBBIES,300,2.970703,2.970703,2.970703,...,1,0,13,6,2,5,2,5,1,CA_1
3,HOBBIES_1_004_CA_1_evaluation,1842,2.0,HOBBIES_1_004,HOBBIES_1,HOBBIES,5,4.640625,4.640625,4.339844,...,1,0,13,6,2,5,2,5,1,CA_1
4,HOBBIES_1_005_CA_1_evaluation,1842,5.0,HOBBIES_1_005,HOBBIES_1,HOBBIES,16,2.880859,3.080078,2.480469,...,1,0,13,6,2,5,2,5,1,CA_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3902715,FOODS_3_823_WI_3_evaluation,1969,NaN,FOODS_3_823,FOODS_3,FOODS,0,2.980469,2.980469,2.480469,...,0,0,19,24,6,5,3,6,1,WI_3
3902716,FOODS_3_824_WI_3_evaluation,1969,NaN,FOODS_3_824,FOODS_3,FOODS,0,2.480469,2.679688,2.000000,...,0,0,19,24,6,5,3,6,1,WI_3
3902717,FOODS_3_825_WI_3_evaluation,1969,NaN,FOODS_3_825,FOODS_3,FOODS,0,3.980469,4.378906,3.980469,...,0,0,19,24,6,5,3,6,1,WI_3
3902718,FOODS_3_826_WI_3_evaluation,1969,NaN,FOODS_3_826,FOODS_3,FOODS,230,1.280273,1.280273,1.280273,...,0,0,19,24,6,5,3,6,1,WI_3


Predict | Day: 1
##########  0.14 min round |  0.14 min total |  39828.47 day sales |
Predict | Day: 2
##########  0.14 min round |  0.28 min total |  36540.38 day sales |
Predict | Day: 3
##########  0.13 min round |  0.40 min total |  37137.73 day sales |
Predict | Day: 4
##########  0.13 min round |  0.53 min total |  36667.58 day sales |
Predict | Day: 5
##########  0.12 min round |  0.66 min total |  41318.20 day sales |
Predict | Day: 6
##########  0.12 min round |  0.77 min total |  49280.06 day sales |
Predict | Day: 7
##########  0.11 min round |  0.89 min total |  49654.29 day sales |
Predict | Day: 8
##########  0.12 min round |  1.00 min total |  43621.84 day sales |
Predict | Day: 9
##########  0.12 min round |  1.12 min total |  37285.05 day sales |
Predict | Day: 10
##########  0.11 min round |  1.23 min total |  41454.63 day sales |
Predict | Day: 11
##########  0.12 min round |  1.35 min total |  42743.03 day sales |
Predict | Day: 12
##########  0.13 min round |  1.48

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.753042,0.688188,0.688751,0.686679,0.768146,0.916658,0.874288,0.901520,0.744240,...,0.826474,0.927827,0.892749,0.766914,0.726412,0.720585,0.724602,0.799029,0.944285,0.716593
1,HOBBIES_1_002_CA_1_evaluation,0.258061,0.179185,0.179749,0.177677,0.265720,0.410616,0.366975,0.287745,0.235237,...,0.313481,0.411217,0.374869,0.268727,0.216486,0.210659,0.214676,0.296547,0.438186,0.371699
2,HOBBIES_1_003_CA_1_evaluation,0.380282,0.306656,0.300634,0.298563,0.386606,0.524676,0.481036,0.410726,0.363467,...,0.435126,0.526038,0.489690,0.391707,0.344716,0.332305,0.336321,0.418192,0.553006,0.486519
3,HOBBIES_1_004_CA_1_evaluation,1.576726,1.512710,1.513273,1.556411,1.859568,2.880035,3.226641,1.765071,1.568762,...,1.872590,2.845898,3.199796,1.546131,1.505628,1.499801,1.549028,1.846013,2.863223,2.817502
4,HOBBIES_1_005_CA_1_evaluation,1.019091,0.945465,0.946028,0.943957,1.054895,1.357839,1.305729,1.071401,1.002277,...,1.044939,1.300725,1.344590,1.057527,1.010536,1.004709,1.008726,1.112624,1.412312,1.343253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0.201094,0.143796,0.124260,0.099562,0.168245,0.314227,0.297308,0.152335,0.143491,...,0.215312,0.408708,0.476055,0.260853,0.243920,0.286225,0.179313,0.206506,0.348027,0.333052
30486,FOODS_3_824_WI_3_evaluation,0.144250,0.065468,0.045932,0.021234,0.074267,0.206200,0.194997,0.519547,0.065163,...,0.109414,0.409556,0.471000,0.175172,0.259158,0.306030,0.089065,0.108346,0.235818,0.224797
30487,FOODS_3_825_WI_3_evaluation,0.799411,0.742113,0.722577,0.696095,0.749129,0.839844,0.828641,0.678052,0.740024,...,0.930880,1.239176,1.278632,0.996482,1.077524,1.094285,0.870279,0.848850,0.960285,0.945454
30488,FOODS_3_826_WI_3_evaluation,1.022139,0.964841,0.945306,0.920607,0.874500,1.169567,1.028474,1.064088,0.964536,...,0.894506,1.180875,1.217752,1.063541,1.085945,1.121424,0.994582,0.914723,1.193525,1.379877


# Export Result

In [11]:
########################### Export
#################################################################################
# Reading competition sample submission and
# merging our predictions
# As we have predictions only for "_validation" data
# we need to do fillna() for "_evaluation" items
submission = pd.read_csv(ORIGINAL/'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv(output_parent/f'submission_v{VER}.csv', index=False)
submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,HOBBIES_1_002_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,HOBBIES_1_003_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,HOBBIES_1_004_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,HOBBIES_1_005_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.201094,0.143796,0.124260,0.099562,0.168245,0.314227,0.297308,0.152335,0.143491,...,0.215312,0.408708,0.476055,0.260853,0.243920,0.286225,0.179313,0.206506,0.348027,0.333052
60976,FOODS_3_824_WI_3_evaluation,0.144250,0.065468,0.045932,0.021234,0.074267,0.206200,0.194997,0.519547,0.065163,...,0.109414,0.409556,0.471000,0.175172,0.259158,0.306030,0.089065,0.108346,0.235818,0.224797
60977,FOODS_3_825_WI_3_evaluation,0.799411,0.742113,0.722577,0.696095,0.749129,0.839844,0.828641,0.678052,0.740024,...,0.930880,1.239176,1.278632,0.996482,1.077524,1.094285,0.870279,0.848850,0.960285,0.945454
60978,FOODS_3_826_WI_3_evaluation,1.022139,0.964841,0.945306,0.920607,0.874500,1.169567,1.028474,1.064088,0.964536,...,0.894506,1.180875,1.217752,1.063541,1.085945,1.121424,0.994582,0.914723,1.193525,1.379877
